# Оконные функции

Оконными называют функции, которые обрабатывают выделенные наборы строк (окна или партиции) и записывают результаты вычислений в отдельном столбце.

Одно из главных преимуществ оконных функций заключается в том, что они возвращают ровно то же количество записей, которое получили на вход.

Представьте, что вы хотите рассчитать некоторое значение для группы строк, объединённых общим признаком (например, id пользователя). Если бы вы воспользовались оператором `GROUP BY`, то на выходе вместо исходного количества строк в группе получили бы одну строку с результатом.

При группировке так происходит всегда — число строк в результирующей таблице всегда равно количеству групп в исходной таблице. В то же время оконная функция позволяет проводить те же расчёты с агрегацией по группам, но при этом сохраняет структуру исходной таблицы — для каждой записи, принадлежащей определённой группе, в отдельном столбце просто указывается результат агрегации.

Определяются окна с помощью оператора `OVER` — в общем виде его синтаксис выглядит так:

>```SQL
>OVER (
>     PARTITION BY column_1, column_2, ...    - определяются партиции внутри окна (аналог GROUP BY) 
>     ORDER BY column_3, ...    - указывается сортировка записей в партициях
>     ROWS/RANGE BETWEEN ...    - задаются границы окна
>)
>```

Для проведения вычислений по заданному в `OVER` окну используются разные функции. Например, с агрегирующей функцией `SUM` запись может выглядеть следующим образом:

>```SQL
>SELECT SUM(column) OVER (PARTITION BY ... ORDER BY ... ROWS/RANGE BETWEEN ...) AS sum
>FROM table
>```

Теперь несколько слов об инструкциях, которые можно указывать при создании окна. Всего их три:

- `PARTITION BY` 
- `ORDER BY` `ASC/DESC`
- `ROWS/RANGE` `BETWEEN`

При этом все они являются необязательными.

Инструкция `PARTITION BY` определяет столбец, по которому данные будут делиться на группы, которые называются партициями. Например, так будет выглядеть группировка по `user_id`:

>```SQL
>SELECT user_id, date, price, 
>       SUM(price) OVER (PARTITION BY user_id) AS sum
>FROM table
>```

Инструкция `ORDER BY` определяет столбец, по которому значения внутри окна будут сортироваться при обработке. Например, сортировка по date внутри окна задаётся так:

>```SQL
>SELECT user_id, date, price, 
>       SUM(price) OVER (PARTITION BY user_id ORDER BY date) AS sum
>FROM table
>```

Почему же считается сумма именно текущей и всех предыдущих, а не вообще всех покупок пользователя?

Дело в том, что при использовании оконных функций в паре с агрегирующими для каждой строки определяется так называемая рамка окна — набор строк в её партиции. Если в `OVER` указать `ORDER BY`, то по умолчанию рамка будет состоять из всех строк от начала партиции до текущей строки (также в рамку будут включены строки, равные текущей строке по значению, указанному в `ORDER BY`).

Именно поэтому в нашем примере сумма считается по каждому пользователю нарастающим итогом.

Если же ORDER BY не указывать, то рамка по умолчанию будет состоять из всех строк партиции, т.е. будет посчитана сумма всех покупок каждого пользователя. Также можно не указывать и `PARTITION BY` — тогда рамкой окна станет вся таблица, и мы просто посчитаем сумму покупок всех пользователей:

>```SQL
>SELECT user_id, date, price, 
>       SUM(price) OVER () AS sum
>FROM table
>```

Инструкции `ROWS` и `RANGE` могут дополнительно задавать границы рамки окна и ограничивать диапазон работы функций внутри партиции. Первым аргументом указывается начало рамки, вторым — конец рамки:


>```SQL
>SELECT user_id, date, price, 
>       SUM(price) OVER (PARTITION BY user_id ORDER BY date ROWS BETWEEN 1 PRECEDING AND CURRENT ROW) AS sum
>FROM table
>```

Рамку можно задать в двух режимах:

- `ROWS` — начало и конец рамки определяются строками относительно текущей строки. 
- `RANGE` — начало и конец рамки задаются разницей значений в столбце из `ORDER BY`.

Начало и конец рамки задаются одним из следующих способов:

>```SQL
>UNBOUNDED PRECEDING
>значение PRECEDING
>CURRENT ROW
>значение FOLLOWING
>UNBOUNDED FOLLOWING
>```

- `UNBOUNDED PRECEDING` — указывает, что рамка начинается с первой строки партиции.
- `UNBOUNDED FOLLOWING` — указывает, что рамка заканчивается на последней строке партиции.
- значение `PRECEDING` и значение `FOLLOWING` — указывают, что рамка начинается или заканчивается со сдвигом на заданное число строк относительно текущей строки.
- `CURRENT ROW` — указывает, что рамка начинается или заканчивается на текущей строке.

Рамка всегда начинается с начала рамки и заканчивается концом рамки. Если конец рамки опущен, подразумевается `CURRENT ROW`. 

По умолчанию рамка определяется так:

>```SQL 
>RANGE UNBOUNDED PRECEDING
>```

Это равносильно расширенному определению рамки:

>```SQL 
>RANGE BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
>```

Варианты значение `PRECEDING` и значение `FOLLOWING` допускаются только в режиме `ROWS`.

Например, следующая запись означает создание рамки, включающей 3 строки до текущей и 3 строки после текущей (разумеется, текущая строка тоже включается в рамку):

>```SQL
>ROWS BETWEEN 3 PRECEDING AND 3 FOLLOWING
>```

Если в инструкции `ORDER BY` находится столбец `date` с типом данных `DATE`, то рамку окна можно задать следующим образом:

>```SQL
>RANGE BETWEEN '3 days' PRECEDING AND '3 days' FOLLOWING
>```

Это будет означать рамку, включающую 3 дня перед и 3 дня после текущей даты (включая текущую дату).

При указании рамки через `RANGE` обязательным условием является указание только одного столбца в инструкции `ORDER BY`.

Как и все остальные инструкции, инструкция `ROWS/RANGE BETWEEN` является необязательной.

Также важно знать, что оконные функции разрешается использовать в запросе только в `SELECT` и `ORDER BY`. Во всех остальных операторах, включая `WHERE`, `HAVING` и `GROUP BY`, они запрещены, так как логически выполняются после обычных агрегирующих функций.

Если необходимо отфильтровать или сгруппировать строки после вычисления оконных функций, можно использовать вложенный запрос:

>```SQL
>SELECT user_id, date, price, sum
>FROM (
>    SELECT user_id, date, price, SUM(price) OVER (PARTITION BY user_id ORDER BY date) AS sum
>    FROM table
>) t
>WHERE sum > 1000
>```

Над результатом оконных функций можно производить разные арифметические операции. Также результат оконных функций может выступать в качестве аргумента других функций:

>```SQL
>SELECT user_id, date, price, 1.15 * SUM(price) OVER (PARTITION BY user_id ORDER BY date) AS sum
>FROM table
>
>
>SELECT user_id, date, price, ROUND(AVG(price) OVER (PARTITION BY user_id ORDER BY date), 2) AS sum
>FROM table
>```

Также при определении инструкций внутри окна можно использовать расчётные поля:

>```SQL
>SELECT user_id, date, price, 
>SUM(price) OVER (PARTITION BY DATE_TRUNC('month', date)) AS monthly_sum
>FROM table
>```

Сами окна можно также определять через оператор WINDOW, а затем вызывать по алиасу в операторе SELECT:

>```SQL
>SELECT SUM(column) OVER w AS sum
>FROM table
>WHERE ...
>GROUP BY ...
>HAVING ...
>WINDOW w AS (
>    PARTITION BY ... 
>    ORDER BY ...
>    ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
>    )
>ORDER BY ...
>LIMIT ...
>```

В паре с оконными функциями могут использоваться функции разных классов:

**1. Агрегирующие функции** `SUM`, `AVG`, `MAX`, `MIN`, `COUNT`

Внутри окна к таким функциям может применяться `ORDER BY`. Так, сортировка позволит получить вместо общей суммы нарастающую, а вместо абсолютного максимума — максимум среди значений вплоть до текущего.

**2. Ранжирующие функции:**

- `ROW_NUMBER` — простая нумерация (1, 2, 3, 4, 5).
- `RANK` — нумерация с учётом повторяющихся значений с пропуском рангов (1, 2, 2, 4, 5).
- `DENSE_RANK` — нумерация с учётом повторяющихся значений без пропуска рангов (1, 2, 2, 3, 4).
Разумеется, для функций ранжирования всегда нужно указывать `ORDER BY`, иначе они будут работать некорректно.

**3. Функции смещения:**

- `LAG`, `LEAD` — значение предыдущей или следующей строки.
- `FIRST_VALUE`, `LAST_VALUE` — первое или последнее значение в окне.

Для функций смещения определение правил сортировки тоже необходимо.

---

### Задача 1.

Примените оконные функции к таблице `products` и с помощью ранжирующих функций упорядочьте все товары по цене — от самых дорогих к самым дешёвым. Добавьте в таблицу следующие колонки:

- Колонку `product_number` с порядковым номером товара (функция `ROW_NUMBER`).
- Колонку `product_rank` с рангом товара с пропусками рангов (функция `RANK`).
- Колонку `product_dense_rank` с рангом товара без пропусков рангов (функция `DENSE_RANK`).

Не забывайте указывать в окне сортировку записей — без неё ранжирующие функции могут давать некорректный результат, если таблица заранее не отсортирована. Деление на партиции внутри окна сейчас не требуется. Сортировать записи в результирующей таблице тоже не нужно.

Поля в результирующей таблице: `product_id`, `name`, `price`, `product_number`, `product_rank`, `product_dense_rank`

```SQL
SELECT 
    product_id,
    name,
    price,
    ROW_NUMBER() OVER(ORDER BY price DESC) AS product_number,
    RANK() OVER(ORDER BY price DESC) AS product_rank,
    DENSE_RANK() OVER(ORDER BY price DESC) AS product_dense_rank
FROM products
```

### Задача 2.

Примените оконную функцию к таблице `products` и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте цену самого дорогого товара. Колонку с этим значением назовите `max_price`. Затем для каждого товара посчитайте долю его цены в стоимости самого дорогого товара — просто поделите одну колонку на другую. Полученные доли округлите до двух знаков после запятой. Колонку с долями назовите `share_of_max`.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `max_price`, `share_of_max`

```SQL
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER() AS max_price,
    ROUND(price / MAX(price) OVER(), 2) AS share_of_max
FROM products
ORDER BY price DESC, product_id ASC
```

### Задача 3.

Примените две оконные функции к таблице `products` — одну с агрегирующей функцией `MAX`, а другую с агрегирующей функцией `MIN` — для вычисления максимальной и минимальной цены. Для двух окон задайте инструкцию `ORDER BY` по убыванию цены. Поместите результат вычислений в две колонки `max_price` и `min_price`.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `max_price`, `min_price`

```SQL
SELECT
    product_id,
    name,
    price,
    MAX(price) OVER(ORDER BY price DESC) AS max_price,
    MIN(price) OVER(ORDER BY price DESC) AS min_price
FROM products
ORDER BY price DESC, product_id ASC
```

### Задача 4.

Сначала на основе таблицы `orders` сформируйте новую таблицу с общим числом заказов по дням. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице `user_actions`). Колонку с днями назовите `date`, а колонку с числом заказов — `orders_count`.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией `SUM` для расчёта накопительной суммы числа заказов. Не забудьте для окна задать инструкцию `ORDER BY` по дате.

Колонку с накопительной суммой назовите `orders_cum_count`. В результате такой операции значение накопительной суммы для последнего дня должно получиться равным общему числу заказов за весь период.

Сортировку результирующей таблицы делать не нужно.

Поля в результирующей таблице: `date`, `orders_count`, `orders_cum_count`

```SQL
SELECT 
    date,
    orders_count,
    SUM(orders_count) OVER(ORDER BY date)::INTEGER AS orders_cum_count
FROM (
    SELECT
        creation_time::DATE AS date,
        COUNT(DISTINCT order_id) AS orders_count
    FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
    GROUP BY creation_time::DATE 
    ORDER BY creation_time::DATE ASC) t
```

### Задача 5.

Для каждого пользователя в таблице `user_actions` посчитайте порядковый номер каждого заказа. Для этого примените оконную функцию `ROW_NUMBER` к колонке с временем заказа. Не забудьте указать деление на партиции по пользователям и сортировку внутри партиций. Отменённые заказы не учитывайте. Новую колонку с порядковым номером заказа назовите `order_number`. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию `order_number`. Добавьте `LIMIT 1000`.

Поля в результирующей таблице: `user_id`, `order_id`, `time`, `order_number`

```SQL
SELECT 
    user_id,
    order_id, 
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
ORDER BY user_id, order_number
LIMIT 1000
```

### Задача 6.

Дополните запрос из предыдущего задания и с помощью оконной функции для каждого заказа каждого пользователя рассчитайте, сколько времени прошло с момента предыдущего заказа. 

Для этого сначала в отдельном столбце с помощью `LAG` сделайте смещение по столбцу `time` на одно значение назад. Столбец со смещёнными значениями назовите `time_lag`. Затем отнимите от каждого значения в колонке `time` новое значение со смещением (либо можете использовать уже знакомую функцию `AGE`). Колонку с полученным интервалом назовите `time_diff`. Менять формат отображения значений не нужно, они должны иметь примерно следующий вид:

>```SQL
>3 days, 12:18:22
>```

По-прежнему не учитывайте отменённые заказы. Также оставьте в запросе порядковый номер каждого заказа, рассчитанный на прошлом шаге. Результат отсортируйте сначала по возрастанию id пользователя, затем по возрастанию порядкового номера заказа. Добавьте `LIMIT 1000`.

Поля в результирующей таблице: `user_id`, `order_id`, `time`, `order_number`, `time_lag`, `time_diff`

```SQL
SELECT 
    user_id,
    order_id, 
    time,
    ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number,
    LAG(time) OVER(PARTITION BY user_id ORDER BY time) AS time_lag,
    AGE(time, LAG(time) OVER(PARTITION BY user_id ORDER BY time)) AS time_diff
FROM user_actions
WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
ORDER BY user_id, order_number
LIMIT 1000
```

### Задача 7.

На основе запроса из предыдущего задания для каждого пользователя рассчитайте, сколько в среднем времени проходит между его заказами. Посчитайте этот показатель только для тех пользователей, которые за всё время оформили более одного неотмененного заказа.

Среднее время между заказами выразите в часах, округлив значения до целого числа. Колонку со средним значением времени назовите `hours_between_orders`. Результат отсортируйте по возрастанию id пользователя.

Добавьте в запрос оператор `LIMIT` и включите в результат только первые 1000 записей.

Поля в результирующей таблице: `user_id`, `hours_between_orders`

```SQL
SELECT 
    user_id,
    ROUND(EXTRACT(epoch FROM AVG(time_diff)) / 3600)::INTEGER AS hours_between_orders
FROM (
    SELECT 
        user_id,
        order_id, 
        time,
        ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY time) AS order_number,
        LAG(time) OVER(PARTITION BY user_id ORDER BY time) AS time_lag,
        AGE(time, LAG(time) OVER(PARTITION BY user_id ORDER BY time)) AS time_diff
    FROM user_actions
    WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
    ORDER BY user_id, order_number) t
WHERE order_number > 1
GROUP BY 1
```

### Задача 8.

Сначала на основе таблицы `orders` сформируйте новую таблицу с общим числом заказов по дням. Вы уже делали это в одной из предыдущих задач. При подсчёте числа заказов не учитывайте отменённые заказы (их можно определить по таблице `user_actions`). Колонку с числом заказов назовите `orders_count`.

Затем поместите полученную таблицу в подзапрос и примените к ней оконную функцию в паре с агрегирующей функцией `AVG` для расчёта скользящего среднего числа заказов. Скользящее среднее для каждой записи считайте по трём предыдущим дням. Подумайте, как правильно задать границы рамки, чтобы получить корректные расчёты.

Полученные значения скользящего среднего округлите до двух знаков после запятой. Колонку с рассчитанным показателем назовите `moving_avg`. Сортировку результирующей таблицы делать не нужно.

Поля в результирующей таблице: `date`, `orders_count`, `moving_avg`

```SQL
SELECT 
    date,
    orders_count,
    ROUND(AVG(orders_count) OVER(ROWS BETWEEN 3 PRECEDING AND 1 PRECEDING), 2) AS moving_avg
FROM    
    (
    SELECT
        creation_time::DATE AS date,
        COUNT(DISTINCT order_id) AS orders_count
    FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM user_actions  WHERE action = 'cancel_order')
    GROUP BY creation_time::DATE 
    ORDER BY creation_time::DATE ASC) t
```

### Задача 9.

Теперь научимся вместе с оконными функциями применять конструкцию CASE. Пример такой записи:

>```SQL
>SELECT
>    CASE
>    WHEN SUM(column) OVER (...) > 100 THEN 'above 100'
>    WHEN SUM(column) OVER (...) < 100 THEN 'below 100'
>    ELSE 'equal 100'
>    END AS sum_case
>FROM table
>```

---

<span style="color:#ff5533">**Задание:**</span>

Отметьте в отдельной таблице тех курьеров, которые доставили в сентябре 2022 года заказов больше, чем в среднем все курьеры.

Сначала для каждого курьера в таблице `courier_actions` рассчитайте общее количество доставленных в сентябре заказов. Затем в отдельном столбце с помощью оконной функции укажите, сколько в среднем заказов доставили в этом месяце все курьеры. После этого сравните число заказов, доставленных каждым курьером, со средним значением в новом столбце. Если курьер доставил больше заказов, чем в среднем все курьеры, то в отдельном столбце с помощью `CASE` укажите число 1, в противном случае укажите 0.

Колонку с результатом сравнения назовите `is_above_avg`, колонку с числом доставленных заказов каждым курьером — `delivered_orders`, а колонку со средним значением — `avg_delivered_orders`. При расчёте среднего значения округлите его до двух знаков после запятой. Результат отсортируйте по возрастанию `id` курьера.

Поля в результирующей таблице: `courier_id`, `delivered_orders`, `avg_delivered_orders`, `is_above_avg`

```SQL
SELECT
    courier_id,
    delivered_orders,
    ROUND(AVG(delivered_orders) OVER(), 2) AS avg_delivered_orders,
    CASE 
        WHEN delivered_orders > AVG(delivered_orders) OVER() THEN 1
        ELSE 0
    END AS is_above_avg
FROM     
    (SELECT 
        courier_id,
        COUNT(order_id) AS delivered_orders
    FROM courier_actions
    WHERE action = 'deliver_order' 
    AND DATE_PART('month', time) = 9 
    AND DATE_PART('year', time) = 2022
    GROUP BY courier_id) t
ORDER BY courier_id
```

### Задача 10.

Если к определению оконной функции добавить предложение `FILTER`, то в окно попадут только те входные строки, для которых условие фильтра будет вычислено как истинное.

При этом предложение `FILTER` допускается только для агрегирующих оконных функций.

В общем виде вся конструкция выглядит так:

>```SQL
>SELECT SUM(column_1) FILTER (WHERE column_2 > 100) OVER (PARTITION BY column_3 ORDER BY column_4) AS sum
>FROM table
>```

--- 

<span style="color:#ff5533">**Задание:**</span>

Примените оконную функцию к таблице `products` и с помощью агрегирующей функции в отдельной колонке для каждой записи проставьте среднюю цену всех товаров. Колонку с этим значением назовите `avg_price`. Затем с помощью оконной функции и оператора `FILTER` в отдельной колонке рассчитайте среднюю цену товаров без учёта самого дорогого. Колонку с этим средним значением назовите `avg_price_filtered`. Полученные средние значения в колонках `avg_price` и `avg_price_filtered` округлите до двух знаков после запятой.

Выведите всю информацию о товарах, включая значения в новых колонках. Результат отсортируйте сначала по убыванию цены товара, затем по возрастанию `id` товара.

Поля в результирующей таблице: `product_id`, `name`, `price`, `avg_price`, `avg_price_filtered`

```SQL
SELECT
    product_id,
    name,
    price,
    ROUND(AVG(price) OVER(), 2) AS avg_price,
    ROUND(AVG(price) FILTER(WHERE price != (SELECT MAX(price) FROM products)) OVER(), 2) AS avg_price_filtered
FROM products
ORDER BY price DESC, product_id
```

### * Задача 11.

Для каждой записи в таблице `user_actions` с помощью оконных функций и предложения `FILTER` посчитайте, сколько заказов сделал и сколько отменил каждый пользователь на момент совершения нового действия.

Иными словами, для каждого пользователя в каждый момент времени посчитайте две накопительные суммы — числа оформленных и числа отменённых заказов. Если пользователь оформляет заказ, то число оформленных им заказов увеличивайте на 1, если отменяет — увеличивайте на 1 количество отмен.

Колонки с накопительными суммами числа оформленных и отменённых заказов назовите соответственно `created_orders` и `canceled_orders`. На основе этих двух колонок для каждой записи пользователя посчитайте показатель `cancel_rate`, т.е. долю отменённых заказов в общем количестве оформленных заказов. Значения показателя округлите до двух знаков после запятой. Колонку с ним назовите `cancel_rate`.

В результате у вас должны получиться три новые колонки с динамическими показателями, которые изменяются во времени с каждым новым действием пользователя.

В результирующей таблице отразите все колонки из исходной таблицы вместе с новыми колонками. Отсортируйте результат по колонкам `user_id`, `order_id`, `time` — по возрастанию значений в каждой.

Добавьте в запрос оператор `LIMIT` и выведите только первые 1000 строк результирующей таблицы.

Поля в результирующей таблице: `user_id`, `order_id`, `action`, `time`, `created_orders`, `canceled_orders`, `cancel_rate`

```SQL
SELECT
    user_id,
    order_id,
    action,
    time,
    COUNT(order_id) FILTER(WHERE action = 'create_order')
        OVER(PARTITION BY user_id ORDER BY time) AS created_orders,
    COUNT(order_id) FILTER(WHERE action = 'cancel_order')
        OVER(PARTITION BY user_id ORDER BY time) AS canceled_orders,
    ROUND(COUNT(order_id) FILTER(WHERE action = 'cancel_order') 
          OVER(PARTITION BY user_id ORDER BY time)::DECIMAL /
    COUNT(order_id) FILTER(WHERE action = 'create_order') 
          OVER(PARTITION BY user_id ORDER BY time), 2) AS cancel_rate
FROM user_actions
ORDER BY user_id, order_id, time
LIMIT 1000
```

### Задача 12.

Из таблицы `courier_actions` отберите топ 10% курьеров по количеству доставленных за всё время заказов. Выведите `id` курьеров, количество доставленных заказов и порядковый номер курьера в соответствии с числом доставленных заказов.

У курьера, доставившего наибольшее число заказов, порядковый номер должен быть равен 1, а у курьера с наименьшим числом заказов — числу, равному десяти процентам от общего количества курьеров в таблице `courier_actions`.

При расчёте номера последнего курьера округляйте значение до целого числа.

Колонки с количеством доставленных заказов и порядковым номером назовите соответственно `orders_count` и `courier_rank`. Результат отсортируйте по возрастанию порядкового номера курьера.

Поля в результирующей таблице: `courier_id`, `orders_count`, `courier_rank` 

```SQL
SELECT *
FROM (
    SELECT
        courier_id, 
        orders_count,
        ROW_NUMBER() OVER(ORDER BY orders_count DESC) AS courier_rank
    FROM (
        SELECT
            courier_id,
            COUNT(order_id) AS orders_count
        FROM courier_actions
        WHERE action = 'deliver_order'
        GROUP BY courier_id
        ORDER BY orders_count DESC, courier_id ASC) t1
    ) t2
WHERE courier_rank <= (SELECT ROUND(0.1 * COUNT(DISTINCT courier_id)) FROM courier_actions)
```

### Задача 13.

С помощью оконной функции отберите из таблицы `courier_actions` всех курьеров, которые работают в нашей компании 10 и более дней. Также рассчитайте, сколько заказов они уже успели доставить за всё время работы.

Будем считать, что наш сервис предлагает самые выгодные условия труда и поэтому за весь анализируемый период ни один курьер не уволился из компании. Возможные перерывы между сменами не учитывайте — для нас важна только разница во времени между первым действием курьера и текущей отметкой времени.

Текущей отметкой времени, относительно которой необходимо рассчитывать продолжительность работы курьера, считайте время последнего действия в таблице `courier_actions`. Учитывайте только целые дни, прошедшие с момента первого выхода курьера на работу (часы и минуты не учитывайте).

В результат включите три колонки: `id` курьера, продолжительность работы в днях и число доставленных заказов. Две новые колонки назовите соответственно `days_employed` и `delivered_orders`. Результат отсортируйте сначала по убыванию количества отработанных дней, затем по возрастанию `id` курьера.

Поля в результирующей таблице: `courier_id`, `days_employed`, `delivered_orders`

```SQL
SELECT
    courier_id,
    days_employed,
    COUNT(order_id) AS delivered_orders
FROM (
    SELECT 
        courier_id,
        DATE_PART('day', AGE((SELECT MAX(time) FROM courier_actions),
            MIN(time) FILTER(WHERE action = 'accept_order')
            OVER(PARTITION BY courier_id ORDER BY time)))::INTEGER AS days_employed,
        order_id, 
        action
    FROM courier_actions
    ) t
WHERE action = 'deliver_order' AND days_employed >= 10
GROUP BY 1, 2
ORDER BY days_employed DESC, courier_id
```

### Задача 14.

На основе информации в таблицах `orders` и `products` рассчитайте стоимость каждого заказа, ежедневную выручку сервиса и долю стоимости каждого заказа в ежедневной выручке, выраженную в процентах. В результат включите следующие колонки: `id` заказа, время создания заказа, стоимость заказа, выручку за день, в который был совершён заказ, а также долю стоимости заказа в выручке за день, выраженную в процентах.

При расчёте долей округляйте их до трёх знаков после запятой.

Результат отсортируйте сначала по убыванию даты совершения заказа (именно даты, а не времени), потом по убыванию доли заказа в выручке за день, затем по возрастанию `id` заказа.

При проведении расчётов отменённые заказы не учитывайте.

Поля в результирующей таблице: `order_id`, `creation_time`, `order_price`, `daily_revenue`, `percentage_of_daily_revenue`

```SQL
WITH 

orders AS (
SELECT 
    order_id,
    creation_time,
    product_ids,
    unnest(product_ids) AS product_id
FROM orders
WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    ),

merge_data AS (
SELECT 
    order_id,
    creation_time,
    product_ids,
    product_id,
    price,
    SUM(price) OVER (PARTITION BY order_id) AS order_price 
FROM orders  
LEFT JOIN products
USING(product_id)
    )


SELECT 
    order_id,
    creation_time,
    order_price,
    SUM(order_price) OVER (PARTITION BY DATE_TRUNC('day', creation_time) ) AS daily_revenue, 
    ROUND(order_price / SUM(order_price) OVER (PARTITION BY DATE_TRUNC('day', creation_time)) * 100 , 3) AS percentage_of_daily_revenue
FROM merge_data
GROUP BY order_id, creation_time, order_price
ORDER BY DATE_TRUNC('day', creation_time) DESC, percentage_of_daily_revenue DESC, order_id
```

### Задача 15.

На основе информации в таблицах `orders` и `products` рассчитайте ежедневную выручку сервиса и отразите её в колонке `daily_revenue`. Затем с помощью оконных функций и функций смещения посчитайте ежедневный прирост выручки. Прирост выручки отразите как в абсолютных значениях, так и в % относительно предыдущего дня. Колонку с абсолютным приростом назовите `revenue_growth_abs`, а колонку с относительным — `revenue_growth_percentage`.

Для самого первого дня укажите прирост равным 0 в обеих колонках. При проведении расчётов отменённые заказы не учитывайте. Результат отсортируйте по колонке с датами по возрастанию.

Метрики `daily_revenue`, `revenue_growth_abs`, `revenue_growth_percentage` округлите до одного знака при помощи `ROUND()`.

Поля в результирующей таблице: `date`, `daily_revenue`, `revenue_growth_abs`, `revenue_growth_percentage`

```SQL
WITH 

orders AS (
SELECT 
    order_id,
    creation_time,
    product_ids,
    unnest(product_ids) AS product_id
FROM orders
WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')
    ),

merge_data AS (
SELECT 
    creation_time::DATE AS date,
    SUM(price) AS order_price 
FROM orders  
LEFT JOIN products
USING(product_id)
GROUP BY 
    creation_time::DATE
    )
    
    
SELECT
    date,
    ROUND(daily_revenue, 1) AS daily_revenue,
    ROUND(COALESCE(daily_revenue - LAG(daily_revenue) OVER(ORDER BY date), 0), 1) AS revenue_growth_abs,
    ROUND(COALESCE((daily_revenue - LAG(daily_revenue) OVER(ORDER BY date)) * 100 / LAG(daily_revenue) OVER(ORDER BY date), 0), 1) AS revenue_growth_percentage
FROM
    (SELECT 
        date,
        SUM(order_price) OVER(PARTITION BY date) AS daily_revenue
    FROM merge_data) t
ORDER BY date ASC
```

### ** Задача 16.

С помощью оконной функции рассчитайте медианную стоимость всех заказов из таблицы `orders`, оформленных в нашем сервисе. В качестве результата выведите одно число. Колонку с ним назовите `median_price`. Отменённые заказы не учитывайте.

Поле в результирующей таблице: `median_price`

```SQL
WITH

ranked_orders AS (
SELECT
    order_id,
    creation_time,
    order_price,
    ROW_NUMBER() OVER(ORDER BY order_price) AS order_rank
FROM 
    (SELECT 
        order_id,
        creation_time,
        SUM(price) AS order_price
    FROM (
        SELECT order_id,
            creation_time,
            product_ids,
            unnest(product_ids) AS product_id
        FROM orders
    WHERE order_id NOT IN (SELECT order_id FROM user_actions WHERE action = 'cancel_order')) t1
    LEFT JOIN products using(product_id)
    GROUP BY 1, 2) t2
)


SELECT 
    CASE 
        WHEN (SELECT MAX(order_rank) FROM ranked_orders) % 2 != 0 
            THEN (SELECT order_price FROM ranked_orders 
                WHERE order_rank = (SELECT AVG(order_rank) FROM ranked_orders))
        WHEN (SELECT MAX(order_rank) FROM ranked_orders) % 2 = 0 
            THEN ((SELECT order_price FROM ranked_orders 
                WHERE order_rank = (SELECT AVG(order_rank) FROM ranked_orders) + 0.5) + 
            (SELECT order_price FROM ranked_orders 
                WHERE order_rank = (SELECT AVG(order_rank) FROM ranked_orders) - 0.5)) / 2
    END AS median_price
```

### Подведём итоги

В этом уроке мы:

- Научились работать с оконными функциями и проводить сложные аналитические расчёты.
- Рассмотрели инструкции <font color=#ff5533>**PARTITION BY**</font>, <font color=#ff5533>**ORDER BY**</font> и <font color=#ff5533>**ROWS BETWEEN**</font>, с помощью которых можно задавать оконные функции и управлять рамкой окна.
- Узнали, как вместе с окнами работают агрегирующие и ранжирующие функции, а также функции смещения.
- Рассмотрели ещё более продвинутый функционал SQL и научились применять оператор <font color=#ff5533>**FILTER**</font> вместе с оконными функциями.
- Узнали, как считаются скользящее среднее и медиана, и решили несколько практических задач, с которыми аналитики часто сталкиваются в своей работе.